In [16]:
import pandas as pd

import re
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import load_model

In [3]:
tweets_binary_df = pd.read_csv('./data/tweets_binary_df.csv')

In [4]:
x_binary = tweets_binary_df['tweet_text']
y_binary = tweets_binary_df['label_n']

In [6]:
#Pre-processing

x_binary = x_binary.apply(lambda z: re.sub(u'http\S+', u'', z)) 
x_binary = x_binary.apply(lambda z: re.sub(u'(\s)@\w+', u'', z))
x_binary = x_binary.apply(lambda z: re.sub(u'#', u'', z))
x_binary = x_binary.apply(lambda z: re.sub(u'RT', u'', z))

In [7]:
tweets_binary_df.head()

,Unnamed: 0,tweet_text,label_n
0,0,Dua's for all those affected by the earthquake...,1
1,1,itvnews: Witness to Nepal #earthquake tells it...,1
2,2,Absolutely devastated by the destruction to my...,1
3,3,Thoughts are with the families in #Nepal,1
4,4,Frightful images! Our prayers echo for everyon...,1


In [12]:
tvec1 = TfidfVectorizer(max_features=10000,ngram_range=(1, 3))
tvec1.fit(x_binary)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [13]:
x_binary_tfidf = tvec1.transform(x_binary)

In [14]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=10000))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [15]:
model.fit(x_binary_tfidf, y_binary, epochs=5, batch_size=256)

Epoch 1/5
38528/38528 [==============================] - 6s 157us/step - loss: 0.3966 - acc: 0.9182
Epoch 2/5
38528/38528 [==============================] - 6s 148us/step - loss: 0.1405 - acc: 0.9594
Epoch 3/5
38528/38528 [==============================] - 6s 154us/step - loss: 0.0929 - acc: 0.9719
Epoch 4/5
38528/38528 [==============================] - 6s 148us/step - loss: 0.0713 - acc: 0.9790
Epoch 5/5
38528/38528 [==============================] - 6s 149us/step - loss: 0.0581 - acc: 0.9827


In [17]:
model.save('binary_classification.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
model = load_model('binary_classification.h5')